In [1]:
import pandas as pd

In [ ]:
import re
import string

In [ ]:
from nltk.corpus import stopwords
import nltk

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
from sklearn.decomposition import NMF

In [2]:
df_union = pd.read_csv('df_union_all.csv')  

/Users/albinasitdikova/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (30) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [ ]:
df_union

## Text preprocessing

In [ ]:
df_union.columns

Delete reviews that contains more non-english words than english, because a lot of french words defined as english

In [ ]:
words = set(nltk.corpus.words.words())

In [ ]:

words = set(nltk.corpus.words.words())

sent = "Io andiamo to the beach with my amico."
" ".join(w for w in nltk.wordpunct_tokenize(sent) \
         if w.lower() in words or not w.isalpha())
# 'Io to the beach with my'

In [ ]:
df_union.locality.value_counts()

In [ ]:
# Compound Term Extraction
repl = lambda x: x.replace('new york', 'new_york').replace('nyc', 'new_york').replace('times square', 'times_square').replace('new york city', 'new_york')#.replace('nyc', 'new_york').replace('nyc', 'new_york').replace('nyc', 'new_york').replace('nyc', 'new_york')

# Text preprocessing steps - remove numbers, captial letters and punctuation
alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())

words = set(nltk.corpus.words.words())
english_only = lambda x: " ".join(w for w in nltk.wordpunct_tokenize(x) \
         if w.lower() in words or not w.isalpha())

df_union['text'] = df_union.text.map(alphanumeric).map(punc_lower).map(repl)#.map(english_only) #.map(repl)
df_union['title'] = df_union.title.map(alphanumeric).map(punc_lower).map(repl)#.map(english_only) #.map(repl)
df_union.head()

In [ ]:
#instead all non-English review will type 0, and return only English words
def is_english(row):
    
    len_review = len(row)
    english_review = " ".join(w for w in nltk.wordpunct_tokenize(row) \
         if w.lower() in words or not w.isalpha())
    
    len_english = len(english_review)
    
    if len_review > len_english * 2:
        return 0
    else:
        return english_review

In [ ]:
df_union.text = df_union.text.apply(is_english)

In [ ]:
df_english = df_union.copy()

In [ ]:
df_english.drop(df_english.loc[df_english['text']==0].index, inplace=True)

In [ ]:
df_english 

In [ ]:
# removing stop words from the text
stop = stopwords.words('english')
stop += ['.', ',', '(', ')', "'", '"']
stop += ['hotel', 'stay']
stop = set(stop)

In [ ]:
df_union

#### Delete reviews where author_id or id id None

In [ ]:
df_english.dropna(subset = ["offering_id"], inplace=True)

In [ ]:
df_english.dropna(subset = ["id"], inplace=True)

In [ ]:
df_english.reset_index(drop=True,inplace=True)

In [ ]:
df_english

## Document-Term Matrix

In [ ]:
# The first document-term matrix has default Count Vectorizer values - counts of unigrams

vectorizer = CountVectorizer(stop_words = stop)

doc_word = vectorizer.fit_transform(df_english.text)

pd.DataFrame(doc_word.toarray(), columns=vectorizer.get_feature_names()).head()

## Topic modeling

### Non-Negative Matrix Factorization (NMF)

In [ ]:
nmf_model = NMF(10)
doc_topic_nmf = nmf_model.fit_transform(doc_word)
doc_topic_nmf.shape

In [ ]:
topic_word_nmf = nmf_model.components_
topic_word_nmf.shape

In [ ]:
words_nmf = vectorizer.get_feature_names()
t = nmf_model.components_.argsort(axis=1)[:,-1:-8:-1]
topic_words_nmf = [[words_nmf[e] for e in l] for l in t]
topic_words_nmf

### Try using TF-IDF instead of Count Vectorizer

In [ ]:
# Create TF-IDF versions of the Count Vectorizers created earlier in the exercise
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf = TfidfVectorizer(stop_words=stop)

In [ ]:
doc_word_tfidf = tfidf.fit_transform(df_english.text)              

In [ ]:
pd.DataFrame(doc_word_tfidf.toarray(), columns=tfidf.get_feature_names()).head()

#### NMF - tf-ifd

In [ ]:
nmf_model_2 = NMF(10)
doc_topic_nmf_2 = nmf_model_2.fit_transform(doc_word_tfidf)
doc_topic_nmf_2.shape

In [ ]:
topic_word_nmf_2 = nmf_model_2.components_
topic_word_nmf_2.shape

In [ ]:
words_2 = tfidf.get_feature_names()
t_2 = nmf_model_2.components_.argsort(axis=1)[:,-1:-7:-1]
topic_words_nmf_2 = [[words_2[e] for e in l] for l in t_2]
topic_words_nmf_2

In [ ]:
doc_topic_nmf_2

In [ ]:
H = pd.DataFrame(doc_topic_nmf_2.round(5),
                 index = df_english.text
                )
H

### !!! try different topic modeling on tf-idf

### LSA

In [ ]:
from sklearn.decomposition import TruncatedSVD

In [ ]:
# Acronynms: Latent Semantic Analysis (LSA) is just another name for 
#  Signular Value Decomposition (SVD) applied to Natural Language Processing (NLP)
lsa = TruncatedSVD(10)
doc_topic_lsa = lsa.fit_transform(doc_word_tfidf)
lsa.explained_variance_ratio_

In [ ]:
doc_topic_lsa

In [ ]:
Vt = pd.DataFrame(doc_topic_lsa.round(5),
             index = df_english.text)
Vt

In [ ]:
words_lsa = tfidf.get_feature_names()
t_lsa = lsa.components_.argsort(axis=1)[:,-1:-7:-1]
topic_words_lsa = [[words_lsa[e] for e in l] for l in t_lsa]
topic_words_lsa

### LDA

### Text Recommendation through Matrix Factorization

In [ ]:
from sklearn.metrics import pairwise_distances

In [ ]:
pairwise_distances(doc_topic_nmf_2[4].reshape(1,-1),doc_topic_nmf_2,metric='cosine')

In [ ]:
pairwise_distances(doc_topic_nmf_2[4].reshape(1,-1),doc_topic_nmf_2,metric='cosine').argsort()

##### You can now take any text and make a recommendation for it:

In [ ]:
t = ['basketball is the best']

In [ ]:
vt = vectorizer.transform(t)

In [ ]:
tt = nmf_model.transform(vt)

In [ ]:
pairwise_distances(tt,doc_topic_nmf_2,metric='cosine').argsort()

In [ ]:
df_english.text[1272]

Simple recommender: list of similar hotels

In [ ]:
from collections import defaultdict

In [ ]:
user_movie_map = defaultdict(list)
hotel_user_map = defaultdict(list)

In [ ]:
user_movie_map

In [ ]:
df_english.columns

In [ ]:
df_english

In [ ]:
len(df_english)

In [ ]:
df_english.loc[0, 'author_id']

In [ ]:
user_movie_map

In [ ]:
df_english.shape[0]

In [ ]:
df_english.head(10)

In [ ]:
df_english.author_id[9]

In [ ]:
for i in range(0, df_english.shape[0]):
#     print(df_english.author_id[i])
#     print(df_english.id[i])
#     print(i)""
    user_movie_map[df_english.author_username[i]].append(df_english.id[i])
    
    hotel_user_map[df_english.id[i]].append(df_english.author_username[i])

In [ ]:
user_movie_map

In [ ]:
def get_similar_hotels(user_movie_map,hotel_user_map,m):
    biglist = []
    for u in hotel_user_map[m]: # get all users that liked this movie
        biglist.extend(user_movie_map[u]) # find all other movies those users liked and add to biglist
    return Counter(biglist).most_common(4)[1:] # use counter to 'count' the other movies that show up most common

In [ ]:
from collections import Counter

In [ ]:
get_similar_hotels(user_movie_map,hotel_user_map,133618731.0)

In [ ]:
hotel_user_map

## Surprise recommender

In [3]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate, train_test_split
from surprise import accuracy
from surprise import Reader
from surprise import BaselineOnly

In [4]:
df_union_all = pd.read_csv('df_union_all.csv')  

/Users/albinasitdikova/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (30) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [ ]:
df_union_all.head()

In [ ]:
df_union_all.columns

In [5]:
df_surprise = df_union_all.loc[:,['id', 'offering_id', 'overall']]

In [ ]:
df_surprise

In [ ]:
#df_surprise['overall'] = pd.to_numeric(df_surprise.overall)

In [ ]:
#use only reviews that > 5


In [6]:
#count reviews for each user
reviews_count_byuser = df_surprise.groupby(['id'])['overall'].count().sort_values(ascending=False)
reviews_count_byuser

id
6562BBD4EA770FE84E579622F68FA181    63
869E5453ED1ECD12EC2E210BEB6B922E    54
6C62B19C4DB8C600576B763C68AF0759    52
85223AA53CB0DE6A6EB7B3C1E99981B1    52
7BC0AF07CC240F2F614A865BBE21B5AA    47
                                    ..
A0C0ABF1FDB3C9DE56BA848DA14F6348     1
A0C0D203DEB1B61000BBF9259A4B5A22     1
A0C0DCCEC004AABAEFB5CD88A6ED4BDA     1
A0C0E290EABB5C946BF70442D7CDE19A     1
7FC56E9337C70B8D630FCFB59E9A988E     1
Name: overall, Length: 576685, dtype: int64

In [7]:
user_more_reviews = reviews_count_byuser[reviews_count_byuser>5]
user_more_reviews.index

Index(['6562BBD4EA770FE84E579622F68FA181', '869E5453ED1ECD12EC2E210BEB6B922E',
       '6C62B19C4DB8C600576B763C68AF0759', '85223AA53CB0DE6A6EB7B3C1E99981B1',
       '7BC0AF07CC240F2F614A865BBE21B5AA', 'F73CC60121EDAB8E6B1637FDE6A09177',
       '17EEAF9A079A2B75E9616F7864F7CA3E', '5A259EBC8C1AEBD6F20D4C48490255FF',
       '106952AB894E1236A6094B030DD73C8F', '5DFE96EC85C67F248DEFFA8B84891A6A',
       ...
       '00BBAC3339F576E164FF9F627489481C', 'FD2BB013EE49D8432414008F8B89C482',
       'FEF59A627074937521F622D5F65794AF', '01ECD34DFB3A71980CFE05D453CB2156',
       'FD2680BF42CF51B2B13A136983B804C4', 'FDB8E88150901C82631CD5548B1A2BB8',
       '027E1165FCD77427C33454661AF677CA', 'FCE4AD28BE08AF16A5B8E91D4D3C4F40',
       'FD502B68BDE687E6E3F59E0578A8B615', 'FFE779A7913FECF3B0476DEB287ADCA8'],
      dtype='object', name='id', length=7252)

In [8]:
df_surprise.id.isin(user_more_reviews.index)

0         False
1         False
2         False
3         False
4         False
          ...  
801189    False
801190    False
801191    False
801192    False
801193    False
Name: id, Length: 801194, dtype: bool

In [9]:
#df_surprise['user'] = df_surprise.id.isin(user_more_reviews.index)

In [10]:
df_surprise = df_surprise[df_surprise.id.isin(user_more_reviews.index)]

In [11]:
df_surprise 

,id,offering_id,overall
35,1005ABAA83773778AB2A4BAAFE443901,93338,5
39,FD711E1ED5A2DA3F69425CDD77A80E5A,93338,5
63,A12D74E5CFC1564773D7C48B490DBE54,93338,5
119,3D693454C428B1D6E5458ACC210F9695,93338,5
120,771E3C17495EE9D080AF87EB2504161F,93338,4
...,...,...,...
801149,0FEEEDDB251FEF3713076F4CBFB08CF7,84117,5
801153,1437F74083AF6DEE8DAAC603F436EDA0,84117,5
801161,657EB42F3264EA15A287F416268A8A68,84117,5
801165,40923C2E4B6283C7450E045801624447,84117,4


In [12]:
#hotel has more than 5 review

In [13]:
reviews_count_by_hotel = df_surprise.groupby(['offering_id'])['overall'].count().sort_values(ascending=False)[:1000]
reviews_count_by_hotel

offering_id
87617      297
111501     284
611947     244
93618      244
93507      240
          ... 
93376       17
73947       17
81170       17
98640       17
1940859     17
Name: overall, Length: 1000, dtype: int64

In [14]:
hotel_more_reviews = reviews_count_by_hotel[reviews_count_by_hotel>1]
hotel_more_reviews.index

Int64Index([  87617,  111501,  611947,   93618,   93507,  249712,   93450,
              81295,   81461,  100570,
            ...
             223154,   81352,  114326,  114583,  123556,   93376,   73947,
              81170,   98640, 1940859],
           dtype='int64', name='offering_id', length=1000)

In [15]:
df_surprise.offering_id.isin(hotel_more_reviews.index)

35        True
39        True
63        True
119       True
120       True
          ... 
801149    True
801153    True
801161    True
801165    True
801167    True
Name: offering_id, Length: 61005, dtype: bool

In [16]:
df_surprise = df_surprise[df_surprise.offering_id.isin(hotel_more_reviews.index)]
df_surprise

,id,offering_id,overall
35,1005ABAA83773778AB2A4BAAFE443901,93338,5
39,FD711E1ED5A2DA3F69425CDD77A80E5A,93338,5
63,A12D74E5CFC1564773D7C48B490DBE54,93338,5
119,3D693454C428B1D6E5458ACC210F9695,93338,5
120,771E3C17495EE9D080AF87EB2504161F,93338,4
...,...,...,...
801149,0FEEEDDB251FEF3713076F4CBFB08CF7,84117,5
801153,1437F74083AF6DEE8DAAC603F436EDA0,84117,5
801161,657EB42F3264EA15A287F416268A8A68,84117,5
801165,40923C2E4B6283C7450E045801624447,84117,4


In [17]:
df_surprise.groupby(['offering_id'])['overall'].count()

offering_id
73792      21
73821      34
73855      56
73943      39
73947      17
           ..
2079052    44
2148819    25
2173604    19
2192761    43
2322597    23
Name: overall, Length: 1000, dtype: int64

# For surprise only review that has more than 5 review for each hotel and more than 5 review from each user?

In [17]:
#df_surprise.to_csv('df_surprise7.csv', index=False)#, header=False)

In [231]:
df_surprise.to_csv('df_surprise.csv', index=False, header=False)

In [232]:
file_path = 'df_surprise.csv'

In [20]:
#file_path = os.path.expanduser('df_surprise.csv')

In [233]:
# As we're loading a custom dataset, we need to define a reader. In the
# movielens-100k dataset, each line has the following format:
# 'user item rating timestamp', separated by '\t' characters.
reader = Reader(line_format='user item rating', sep=',')

In [234]:
data = Dataset.load_from_file(file_path, reader=reader)

In [23]:
# We can now use this dataset as we please, e.g. calling cross_validate
cross_validate(BaselineOnly(), data, verbose=True)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating RMSE, MAE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8801  0.8658  0.8612  0.8795  0.8782  0.8730  0.0079  
MAE (testset)     0.6821  0.6715  0.6699  0.6827  0.6804  0.6773  0.0055  
Fit time          0.17    0.27    0.30    0.46    0.25    0.29    0.10    
Test time         0.20    0.10    0.15    0.21    0.23    0.18    0.05    


{'test_rmse': array([0.88011692, 0.8657763 , 0.86120283, 0.87946574, 0.87822799]),
 'test_mae': array([0.68214641, 0.67150266, 0.66994203, 0.68270039, 0.68044373]),
 'fit_time': (0.17212224006652832,
  0.2737891674041748,
  0.2969951629638672,
  0.46321678161621094,
  0.2503221035003662),
 'test_time': (0.20330095291137695,
  0.09873008728027344,
  0.15319514274597168,
  0.213547945022583,
  0.22829484939575195)}

In [ ]:
# We'll use the famous SVD algorithm.
algo = SVD(verbose=True)

# you can also build KNNBasic and other types of models

# Run 5-fold cross-validation and print results
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, n_jobs=-1, verbose=True)

# ml-100k dataset: this takes around .5 minute
# jester dataset: this takes around 10 minutes

In [ ]:
# let's do train-test-split, where test set is 25% of the ratings
trainset, testset = train_test_split(data, test_size=.25)

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# you can also use this one-liner: `predictions = algo.fit(trainset).test(testset)`

In [ ]:
# compute RMSE
accuracy.rmse(predictions)

In [ ]:
# get a prediction for specific users and items.
uid = '8C0B42FF3C0FA366A21CFD785302A032'
iid = '84117'

pred = algo.predict(uid, iid, verbose=True)

In [ ]:
df_surprise[df_surprise.id=='8C0B42FF3C0FA366A21CFD785302A032']

In [ ]:
df_surprise.offering_id.value_counts()

In [ ]:
df_surprise[df_surprise.offering_id==3337599]

In [ ]:
df_surprise[df_surprise.id=='3FE29CCADCF6AB444B131848DE3E7759']

In [ ]:
# get a prediction for specific users and items.
uid = '8C0B42FF3C0FA366A21CFD785302A032'
iid = 3337599

pred = algo.predict(uid, iid, verbose=True)

In [ ]:
uid = '22'
iid = 22

pred = algo.predict(uid, iid, verbose=True)

### WHY imposibble FALSE??? If new user and new hotel?

In [23]:
from collections import defaultdict

from surprise import SVD
from surprise import Dataset

In [24]:
# First train an SVD algorithm on the movielens dataset.
trainset = data.build_full_trainset()
algo = SVD()
algo.fit(trainset)

In [25]:
# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

In [26]:
def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [27]:
top_n = get_top_n(predictions, n=10)

In [28]:
# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

1005ABAA83773778AB2A4BAAFE443901 ['1152288', '1516481', '87632', '111408', '288617', '84117', '1510002', '98726', '2322597', '93614']
FD711E1ED5A2DA3F69425CDD77A80E5A ['781627', '89585', '1152288', '674283', '122703', '576787', '1027494', '224783', '2322597', '93566']
A12D74E5CFC1564773D7C48B490DBE54 ['674283', '1456416', '1152288', '2322597', '1027494', '576787', '84090', '84131', '107836', '225105']
3D693454C428B1D6E5458ACC210F9695 ['576787', '1516481', '1152288', '80983', '225105', '674283', '1028569', '613399', '93566', '1737951']
771E3C17495EE9D080AF87EB2504161F ['93566', '225105', '1516481', '674283', '80983', '1152288', '2322597', '224224', '576787', '1737951']
7788F6DEA6D525F6A79EC6208AE8C793 ['80983', '1516481', '224224', '122703', '1152288', '576787', '93566', '1175274', '113298', '1762915']
4F127FBECCC472FC8109EE9DA46FB8BE ['2015227', '674283', '84090', '2322597', '1199588', '98726', '217770', '576787', '841981', '224783']
6B525E7772D5BCEE61CC6A282448B360 ['674283', '576787'

In [67]:
top_1 = top_n['1005ABAA83773778AB2A4BAAFE443901']

In [70]:
type(top_n)

collections.defaultdict

In [ ]:
df_union_all

In [71]:
with open('result_1.txt', 'w') as outfile:
    json.dump(top_1, outfile)

In [34]:
top_n.to_csv('top_10_rec.csv')

AttributeError: 'collections.defaultdict' object has no attribute 'to_csv'

In [41]:
import json

In [42]:
with open('result.txt', 'w') as outfile:
    json.dump(top_n, outfile)

In [65]:
len(top_n[0:100])

TypeError: unhashable type: 'slice'

In [56]:
with open('result.txt') as json_file:
    top_n_2 = json.load(json_file)

In [57]:
top_n_2['1005ABAA83773778AB2A4BAAFE443901']

[['1152288', 4.634832331693354],
 ['1516481', 4.617369391485515],
 ['87632', 4.614475474021299],
 ['111408', 4.608381035291188],
 ['288617', 4.602859699887558],
 ['84117', 4.580249168544883],
 ['1510002', 4.569049589056493],
 ['98726', 4.549953031110445],
 ['2322597', 4.546738913830664],
 ['93614', 4.54317794017153]]

In [62]:
list_keys = list(top_n_2.keys())

In [63]:
list_keys

['1005ABAA83773778AB2A4BAAFE443901',
 'FD711E1ED5A2DA3F69425CDD77A80E5A',
 'A12D74E5CFC1564773D7C48B490DBE54',
 '3D693454C428B1D6E5458ACC210F9695',
 '771E3C17495EE9D080AF87EB2504161F',
 '7788F6DEA6D525F6A79EC6208AE8C793',
 '4F127FBECCC472FC8109EE9DA46FB8BE',
 '6B525E7772D5BCEE61CC6A282448B360',
 'A3BEC98880E7420DE9B7D59932C9B75D',
 '29491F39069C9B01A593F4C600F13F4E',
 'C24107F1947A5BA006BB2DF560CFA399',
 '7305BC9B17FEBB9565560954E6241F16',
 '25E1AB0EE4A20CC2A01C6CA042E9BB5D',
 'FF74012E2686AEB658CE9B0D85CAD372',
 'C1768E4875BEFA5944E5C27D7F469B52',
 'FB5E8F61366A87FFFEF71782E79D77D3',
 'D65F1F963BCE4018B152856B6CFF59DF',
 'D562000DBBC2BA10808315B117228C4B',
 '9B3E1976F9D19D50C8F15F92C6BBC3D9',
 '87AA5304C8FD3951FDA3B5BCDF96350F',
 '220E76FE87C1D9B09A67DAEDC8852E99',
 '1CBCD567F43D76EB6F22B429BCA5D5C1',
 'EC134ACB40E825C6BA2A4C095DDE7A59',
 '0269DD15539962B0A3AFF16EDBFA6326',
 '05FCFBC8F7E64865FC99ACE466DB958F',
 '14EA4C80C1713DB151CCE2AE7572D445',
 '04820B2949C24634019BE35A670D43A3',
 

In [85]:
top_n_3 = []
for uid, user_ratings in top_n.items():
    top_n_3.append(uid, [iid for (iid, _) in user_ratings])

TypeError: append() takes exactly one argument (2 given)

## k nearest neighbors of item

In [186]:
import io  # needed because of weird encoding of u.item file

from surprise import KNNBaseline
from surprise import Dataset
from surprise import get_dataset_dir

In [297]:
df_offering = df_offering.rename(columns={'id': 'offering_id'})

In [298]:
type(df_offering.offering_id[0])
df_offering['offering_id'] = df_offering['offering_id'].astype(int)

In [299]:
df_offering_small = pd.merge(df_surprise, df_offering,how="inner", on=["offering_id"])

In [300]:
df_offering_small.drop(columns=['id', 'overall'], inplace=True)

In [301]:
df_offering_small.drop_duplicates(inplace=True)

In [302]:
df_offering_small = df_offering_small.rename(columns={'offering_id': 'id'})

In [303]:
df_offering_small

,id,hotel_class,region_id,url,phone,details,type,name,region,street-address,postal-code,locality
0,93338,3.0,60763,http://www.tripadvisor.com/Hotel_Review-g60763...,NaN,NaN,hotel,Hotel Beacon,NY,2130 Broadway at 75th Street,10023,New York City
56,1762573,4.0,60763,http://www.tripadvisor.com/Hotel_Review-g60763...,NaN,NaN,hotel,Andaz 5th Avenue,NY,485 5th Avenue,10017,New York City
118,1776857,5.0,60763,http://www.tripadvisor.com/Hotel_Review-g60763...,NaN,NaN,hotel,"The Setai Fifth Avenue, a Capella managed Hotel",NY,400 Fifth Avenue,10018,New York City
151,1641016,5.0,60763,http://www.tripadvisor.com/Hotel_Review-g60763...,NaN,NaN,hotel,The Chatwal,NY,130 West 44th Street,10036,New York City
181,1456560,4.0,60763,http://www.tripadvisor.com/Hotel_Review-g60763...,NaN,NaN,hotel,Eventi - a Kimpton Hotel,NY,851 Avenue of the Americas (Sixth Avenue),10001,New York City
...,...,...,...,...,...,...,...,...,...,...,...,...
49542,84065,5.0,28970,http://www.tripadvisor.com/Hotel_Review-g28970...,NaN,NaN,hotel,Four Seasons Washington D.C.,DC,2800 Pennsylvania Ave. NW,20007,Washington DC
49572,84090,5.0,28970,http://www.tripadvisor.com/Hotel_Review-g28970...,NaN,NaN,hotel,Park Hyatt Washington,DC,1201 24th Street NW,20037,Washington DC
49646,235513,4.5,28970,http://www.tripadvisor.com/Hotel_Review-g28970...,NaN,NaN,hotel,Sofitel Washington DC,DC,806 15th Street NW,20005,Washington DC
49763,84093,4.0,28970,http://www.tripadvisor.com/Hotel_Review-g28970...,NaN,NaN,hotel,"Palomar Washington DC, a Kimpton Hotel",DC,2121 P Street NW,20037,Washington DC


In [304]:
def read_item_names(df_offering_small):
    """Read the u.item file from MovieLens 100-k dataset and return two
    mappings to convert raw ids into movie names and movie names into raw ids.
    """

    rid_to_name = {}
    name_to_rid = {}
    
    rid_to_name = pd.Series(df_offering_small.name.values,index=df_offering_small.id).to_dict()
    name_to_rid = pd.Series(df_offering_small.id.values,index=df_offering_small.name).to_dict()

    return rid_to_name, name_to_rid

In [305]:
# First, train the algortihm to compute the similarities between items
data_knn = Dataset.load_from_file(file_path, reader=reader)
trainset_knn = data.build_full_trainset()
sim_options_knn = {'name': 'pearson_baseline', 'user_based': False}
algo_knn = KNNBaseline(sim_options=sim_options_knn)
algo_knn.fit(trainset_knn)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [306]:
# Read the mappings raw id <-> movie name
rid_to_name, name_to_rid = read_item_names(df_offering_small)

In [312]:
name_to_rid.keys()

dict_keys(['Hotel Beacon', 'Andaz 5th Avenue', 'The Setai Fifth Avenue, a Capella managed Hotel', 'The Chatwal', 'Eventi - a Kimpton Hotel', 'The Iroquois', 'San Carlos Hotel', 'Hilton Americas - Houston', 'Hilton Garden Inn Houston Energy Corridor', 'Four Seasons Hotel Houston', 'Hotel Icon, Autograph Collection', 'Omni Houston Hotel', 'Alden-Houston', 'Westin Galleria Houston Hotel', 'Renaissance Houston Hotel Greenway Plaza', 'Hampton Inn Houston - Near The Galleria', 'Marriott Houston Airport at George Bush Intercontinental', 'Holiday Inn Houston Intercontinental Airport', 'Sheraton North Houston at George Bush Intercontinental', 'DoubleTree by Hilton Houston Hobby Airport', 'Hilton Houston Southwest', 'Four Points by Sheraton Houston Southwest', 'Sheraton Suites Houston Near The Galleria', 'DoubleTree Suites by Hilton Houston by the Galleria', 'Hilton Houston Post Oak', 'Houston Marriott Medical Center', 'Doubletree Hotel Houston Downtown', 'Houston Marriott North', 'Westin Oaks H

In [365]:
list_knn=[]
dict_hotel = {}
for name_hotel in name_to_rid.keys():
    # Retrieve inner id of the movie Toy Story
    hotel_raw_id = str(name_to_rid[name_hotel])
    hotel_inner_id = algo_knn.trainset.to_inner_iid(hotel_raw_id)
    
    # Retrieve inner ids of the nearest neighbors of Toy Story.
    hotel_neighbors = algo_knn.get_neighbors(hotel_inner_id, k=10)
    
     # Convert inner ids of the neighbors into names.
    hotel_neighbors = (algo_knn.trainset.to_raw_iid(inner_id)
                       for inner_id in hotel_neighbors)
    
    hotel_neighbors = (rid_to_name[int(rid)]
                       for rid in hotel_neighbors)
    
    dict_hotel[name_hotel] = list(hotel_neighbors)
    
    list_knn.append(dict_hotel)

In [367]:
dict_hotel['Hotel Beacon']

['Hudson New York',
 'Sir Francis Drake Hotel - a Kimpton Hotel',
 'Omni Hotel at Independence Park',
 'Doubletree Hotel Metropolitan - New York City',
 'The Langham, Boston',
 'Hotel Monaco Chicago - a Kimpton Hotel',
 'Renaissance Washington, DC Hotel',
 'The Boston Park Plaza Hotel & Towers',
 'Hilton New York',
 'Omni Parker House']

In [368]:
df_knn = pd.DataFrame.from_dict(dict_hotel, orient='index',columns=['r0','r1', 'r2', 'r3', 'r4', 'r5', 'r6', 'r7', 'r8', 'r9'])

In [371]:
df_knn

,r0,r1,r2,r3,r4,r5,r6,r7,r8,r9
Hotel Beacon,Hudson New York,Sir Francis Drake Hotel - a Kimpton Hotel,Omni Hotel at Independence Park,Doubletree Hotel Metropolitan - New York City,"The Langham, Boston",Hotel Monaco Chicago - a Kimpton Hotel,"Renaissance Washington, DC Hotel",The Boston Park Plaza Hotel & Towers,Hilton New York,Omni Parker House
Andaz 5th Avenue,Four Seasons Hotel San Francisco,Andaz San Diego,Grand Hyatt New York,Hyatt At Olive 8,Conrad Chicago,Manchester Grand Hyatt San Diego,"Mandarin Oriental, Washington D.C.",Hudson New York,Grand Hyatt Washington,Hyatt Regency Denver At Colorado Convention Ce...
"The Setai Fifth Avenue, a Capella managed Hotel",Waldorf Astoria Chicago,The Grand Del Mar,Soho Grand Hotel,Four Seasons Hotel New York,"The Standard, East Village",The St. Regis New York,XV Beacon,Ritz-Carlton Dallas,The London NYC,The Peninsula Chicago
The Chatwal,The St. Regis Houston,"Mandarin Oriental, New York",Sofitel Chicago Water Tower,"St. Regis Hotel, San Francisco",W New York - Union Square,Hilton New York,W Washington DC,"Algonquin Hotel Times Square, Autograph Collec...",Trump International Hotel & Tower Chicago,Ritz-Carlton Battery Park
Eventi - a Kimpton Hotel,Hotel Solamar - a Kimpton Hotel,The Muse Hotel New York,New York Marriott Marquis,Nine Zero Hotel - a Kimpton Hotel,Hotel Monaco Seattle - a Kimpton Hotel,"Monaco Baltimore, a Kimpton Hotel",Hotel Palomar San Francisco - a Kimpton Hotel,Crowne Plaza Times Square Manhattan,Grand Hyatt New York,Andaz 5th Avenue
...,...,...,...,...,...,...,...,...,...,...
Four Seasons Washington D.C.,"Mandarin Oriental, Washington D.C.",Four Seasons Hotel Boston,Four Seasons Baltimore,Crowne Plaza Times Square Manhattan,Loews Regency Hotel,Doubletree Guest Suites Times Square,The Normandy Hotel,The Hay-Adams,Park Hyatt Washington,Park Hyatt Chicago
Park Hyatt Washington,Ritz-Carlton Battery Park,Waldorf Astoria New York,Hilton New York,The New York Palace Hotel,Le Meridien Philadelphia,Omni Charlotte Hotel,Sofitel Philadelphia Hotel,DoubleTree Suites by Hilton - Boston,Boston Harbor Hotel,Grand Hyatt DFW
Sofitel Washington DC,Sofitel Chicago Water Tower,Sofitel New York,Sofitel Philadelphia Hotel,Novotel New York Times Square,"Palomar Washington DC, a Kimpton Hotel",Le Parker Meridien,The Fairmont Washington DC,Capital Hilton,Grand Hyatt Seattle,The Sofia Hotel
"Palomar Washington DC, a Kimpton Hotel","Renaissance Washington, DC Dupont Circle Hotel",Sofitel Washington DC,"The Liaison Capitol Hill, An Affinia Hotel",Hotel Monaco Chicago - a Kimpton Hotel,"The Westin Georgetown, Washington D.C.","Topaz, a Kimpton Hotel",Hotel Monaco Denver - a Kimpton Hotel,Hotel Solamar - a Kimpton Hotel,Sheraton Boston Hotel,"Monaco Washington DC, a Kimpton Hotel"


In [370]:
df_knn.to_csv('df_knn.csv', index=True, index_label = 'id')

In [238]:
# Retrieve inner id of the movie Toy Story
hotel_raw_id = name_to_rid['Hotel Beacon']
hotel_inner_id = algo_knn.trainset.to_inner_iid(hotel_raw_id)

# Retrieve inner ids of the nearest neighbors of Toy Story.
hotel_neighbors = algo_knn.get_neighbors(hotel_inner_id, k=10)

# Convert inner ids of the neighbors into names.
hotel_neighbors = (algo_knn.trainset.to_raw_iid(inner_id)
                       for inner_id in hotel_neighbors)
hotel_neighbors = (rid_to_name[rid]
                       for rid in hotel_neighbors)

print()
print('The 10 nearest neighbors of Hotel Beacon are:')
for hotel in hotel_neighbors:
    print(hotel)


The 10 nearest neighbors of Hotel Beacon are:
Hudson New York
Sir Francis Drake Hotel - a Kimpton Hotel
Omni Hotel at Independence Park
Doubletree Hotel Metropolitan - New York City
The Langham, Boston
Hotel Monaco Chicago - a Kimpton Hotel
Renaissance Washington, DC Hotel
The Boston Park Plaza Hotel & Towers
Hilton New York
Omni Parker House


In [225]:
df_surprise

,id,offering_id,overall
35,1005ABAA83773778AB2A4BAAFE443901,93338,5
39,FD711E1ED5A2DA3F69425CDD77A80E5A,93338,5
63,A12D74E5CFC1564773D7C48B490DBE54,93338,5
119,3D693454C428B1D6E5458ACC210F9695,93338,5
120,771E3C17495EE9D080AF87EB2504161F,93338,4
...,...,...,...
801149,0FEEEDDB251FEF3713076F4CBFB08CF7,84117,5
801153,1437F74083AF6DEE8DAAC603F436EDA0,84117,5
801161,657EB42F3264EA15A287F416268A8A68,84117,5
801165,40923C2E4B6283C7450E045801624447,84117,4


In [226]:
df_offering[df_offering.id=='93338']

,hotel_class,region_id,url,phone,details,type,id,name,region,street-address,postal-code,locality
15,3.0,60763,http://www.tripadvisor.com/Hotel_Review-g60763...,NaN,NaN,hotel,93338,Hotel Beacon,NY,2130 Broadway at 75th Street,10023,New York City


App result:

In [72]:
top_n

defaultdict(list,
            {'1005ABAA83773778AB2A4BAAFE443901': [('1152288',
               4.634832331693354),
              ('1516481', 4.617369391485515),
              ('87632', 4.614475474021299),
              ('111408', 4.608381035291188),
              ('288617', 4.602859699887558),
              ('84117', 4.580249168544883),
              ('1510002', 4.569049589056493),
              ('98726', 4.549953031110445),
              ('2322597', 4.546738913830664),
              ('93614', 4.54317794017153)],
             'FD711E1ED5A2DA3F69425CDD77A80E5A': [('781627',
               4.985515736037497),
              ('89585', 4.929455435606509),
              ('1152288', 4.863086566532487),
              ('674283', 4.827432935023497),
              ('122703', 4.821960166397605),
              ('576787', 4.81944831594815),
              ('1027494', 4.782173671527035),
              ('224783', 4.7557262701678225),
              ('2322597', 4.754980583174056),
              ('93566',

In [112]:
df_top_n = pd.DataFrame.from_dict(top_n, orient='index',columns=['r0','r1', 'r2', 'r3', 'r4', 'r5', 'r6', 'r7', 'r8', 'r9'])

In [113]:
df_top_n

,r0,r1,r2,r3,r4,r5,r6,r7,r8,r9
1005ABAA83773778AB2A4BAAFE443901,"(1152288, 4.634832331693354)","(1516481, 4.617369391485515)","(87632, 4.614475474021299)","(111408, 4.608381035291188)","(288617, 4.602859699887558)","(84117, 4.580249168544883)","(1510002, 4.569049589056493)","(98726, 4.549953031110445)","(2322597, 4.546738913830664)","(93614, 4.54317794017153)"
FD711E1ED5A2DA3F69425CDD77A80E5A,"(781627, 4.985515736037497)","(89585, 4.929455435606509)","(1152288, 4.863086566532487)","(674283, 4.827432935023497)","(122703, 4.821960166397605)","(576787, 4.81944831594815)","(1027494, 4.782173671527035)","(224783, 4.7557262701678225)","(2322597, 4.754980583174056)","(93566, 4.743958909451692)"
A12D74E5CFC1564773D7C48B490DBE54,"(674283, 5)","(1456416, 5)","(1152288, 5)","(2322597, 4.988932411118821)","(1027494, 4.9814470206972254)","(576787, 4.970225536020592)","(84090, 4.945153119568745)","(84131, 4.909148246847366)","(107836, 4.903174609118013)","(225105, 4.882768662906131)"
3D693454C428B1D6E5458ACC210F9695,"(576787, 4.9949463413402695)","(1516481, 4.9637567141764025)","(1152288, 4.9528678224114815)","(80983, 4.9435141497153445)","(225105, 4.9198128447141265)","(674283, 4.911947314862596)","(1028569, 4.904498252459152)","(613399, 4.901536611591851)","(93566, 4.887707498821687)","(1737951, 4.875732218629177)"
771E3C17495EE9D080AF87EB2504161F,"(93566, 5)","(225105, 5)","(1516481, 5)","(674283, 5)","(80983, 5)","(1152288, 5)","(2322597, 5)","(224224, 4.996409818485652)","(576787, 4.995546189024599)","(1737951, 4.9939351142144135)"
...,...,...,...,...,...,...,...,...,...,...
86AA28E95FF56349EB9B4309E292C294,"(1152288, 4.73610435672495)","(93566, 4.718734829345662)","(2322597, 4.601109252641365)","(1028569, 4.598362434679479)","(84117, 4.595219087689733)","(674283, 4.594456053114328)","(613399, 4.58115288301719)","(112412, 4.574820391899998)","(1516481, 4.574226364347007)","(1199588, 4.568993044430985)"
9EC19CA14621FBFFF3A486BEDEF0876D,"(1152288, 4.781133826213233)","(84117, 4.733012370701169)","(93614, 4.697685240017122)","(1199588, 4.688577918042726)","(674283, 4.675536297362828)","(84065, 4.674782478612043)","(87632, 4.655976698256063)","(93566, 4.653041173425564)","(288617, 4.647449228243611)","(89585, 4.645246814787161)"
E810C7DE395924F731A5E445D4B52D5C,"(1152288, 4.883514994926274)","(89585, 4.832597630254901)","(576787, 4.758428235621609)","(107836, 4.751701991484981)","(1566945, 4.728864815154375)","(1516481, 4.713472415183931)","(208452, 4.704771220431806)","(112273, 4.696289523439903)","(1742256, 4.6690884757078415)","(80983, 4.653060603452109)"
0,None,None,None,None,None,None,None,None,None,None


In [114]:
df_top_n.dropna(inplace=True)

In [115]:
df_top_n['r0']

1005ABAA83773778AB2A4BAAFE443901    (1152288, 4.634832331693354)
FD711E1ED5A2DA3F69425CDD77A80E5A     (781627, 4.985515736037497)
A12D74E5CFC1564773D7C48B490DBE54                     (674283, 5)
3D693454C428B1D6E5458ACC210F9695    (576787, 4.9949463413402695)
771E3C17495EE9D080AF87EB2504161F                      (93566, 5)
                                                ...             
BEDA3909A6EE531E14353DF03C415AF3                     (224224, 5)
BF7773D1C56DF2F66F695DF660EDD70E     (246937, 4.695758717828254)
86AA28E95FF56349EB9B4309E292C294     (1152288, 4.73610435672495)
9EC19CA14621FBFFF3A486BEDEF0876D    (1152288, 4.781133826213233)
E810C7DE395924F731A5E445D4B52D5C    (1152288, 4.883514994926274)
Name: r0, Length: 7194, dtype: object

In [116]:
df_top_n.loc['1005ABAA83773778AB2A4BAAFE443901']

r0    (1152288, 4.634832331693354)
r1    (1516481, 4.617369391485515)
r2      (87632, 4.614475474021299)
r3     (111408, 4.608381035291188)
r4     (288617, 4.602859699887558)
r5      (84117, 4.580249168544883)
r6    (1510002, 4.569049589056493)
r7      (98726, 4.549953031110445)
r8    (2322597, 4.546738913830664)
r9       (93614, 4.54317794017153)
Name: 1005ABAA83773778AB2A4BAAFE443901, dtype: object

In [117]:
df_top_n.columns

Index(['r0', 'r1', 'r2', 'r3', 'r4', 'r5', 'r6', 'r7', 'r8', 'r9'], dtype='object')

In [118]:
df_top_n['r0']=df_top_n.r0.map(lambda x: x[0])
df_top_n['r1']=df_top_n.r1.map(lambda x: x[0])
df_top_n['r2']=df_top_n.r2.map(lambda x: x[0])
df_top_n['r3']=df_top_n.r3.map(lambda x: x[0])
df_top_n['r4']=df_top_n.r4.map(lambda x: x[0])
df_top_n['r5']=df_top_n.r5.map(lambda x: x[0])
df_top_n['r6']=df_top_n.r6.map(lambda x: x[0])
df_top_n['r7']=df_top_n.r7.map(lambda x: x[0])
df_top_n['r8']=df_top_n.r8.map(lambda x: x[0])
df_top_n['r9']=df_top_n.r9.map(lambda x: x[0])

In [119]:
df_top_n

,r0,r1,r2,r3,r4,r5,r6,r7,r8,r9
1005ABAA83773778AB2A4BAAFE443901,1152288,1516481,87632,111408,288617,84117,1510002,98726,2322597,93614
FD711E1ED5A2DA3F69425CDD77A80E5A,781627,89585,1152288,674283,122703,576787,1027494,224783,2322597,93566
A12D74E5CFC1564773D7C48B490DBE54,674283,1456416,1152288,2322597,1027494,576787,84090,84131,107836,225105
3D693454C428B1D6E5458ACC210F9695,576787,1516481,1152288,80983,225105,674283,1028569,613399,93566,1737951
771E3C17495EE9D080AF87EB2504161F,93566,225105,1516481,674283,80983,1152288,2322597,224224,576787,1737951
...,...,...,...,...,...,...,...,...,...,...
BEDA3909A6EE531E14353DF03C415AF3,224224,113298,674283,1485961,112273,781627,80983,576787,1175274,246937
BF7773D1C56DF2F66F695DF660EDD70E,246937,113298,2322597,1199588,674283,308759,878358,225105,576787,1456416
86AA28E95FF56349EB9B4309E292C294,1152288,93566,2322597,1028569,84117,674283,613399,112412,1516481,1199588
9EC19CA14621FBFFF3A486BEDEF0876D,1152288,84117,93614,1199588,674283,84065,87632,93566,288617,89585


In [140]:
df_top_n.to_csv('df_top_n.csv', index=True, index_label = 'user_id')

In [141]:
df_top_n.index

Index(['1005ABAA83773778AB2A4BAAFE443901', 'FD711E1ED5A2DA3F69425CDD77A80E5A',
       'A12D74E5CFC1564773D7C48B490DBE54', '3D693454C428B1D6E5458ACC210F9695',
       '771E3C17495EE9D080AF87EB2504161F', '7788F6DEA6D525F6A79EC6208AE8C793',
       '4F127FBECCC472FC8109EE9DA46FB8BE', '6B525E7772D5BCEE61CC6A282448B360',
       'A3BEC98880E7420DE9B7D59932C9B75D', '29491F39069C9B01A593F4C600F13F4E',
       ...
       '56CD40E3365FE8712D8220639896C203', 'B5B8B54A7E9C4ED3E652AFC8B2D94A2B',
       '5AB42DB530F1F7C414D239F39D92E1AB', '51C55763B7314F20B3C209B33F0B725D',
       'CBF58BB5CBCB3282CCE7FCE281ED5352', 'BEDA3909A6EE531E14353DF03C415AF3',
       'BF7773D1C56DF2F66F695DF660EDD70E', '86AA28E95FF56349EB9B4309E292C294',
       '9EC19CA14621FBFFF3A486BEDEF0876D', 'E810C7DE395924F731A5E445D4B52D5C'],
      dtype='object', length=7194)

In [142]:
df_top_n_read = pd.read_csv('df_top_n.csv')  

In [144]:
df_top_n_read.user_id

0       1005ABAA83773778AB2A4BAAFE443901
1       FD711E1ED5A2DA3F69425CDD77A80E5A
2       A12D74E5CFC1564773D7C48B490DBE54
3       3D693454C428B1D6E5458ACC210F9695
4       771E3C17495EE9D080AF87EB2504161F
                      ...               
7189    BEDA3909A6EE531E14353DF03C415AF3
7190    BF7773D1C56DF2F66F695DF660EDD70E
7191    86AA28E95FF56349EB9B4309E292C294
7192    9EC19CA14621FBFFF3A486BEDEF0876D
7193    E810C7DE395924F731A5E445D4B52D5C
Name: user_id, Length: 7194, dtype: object

In [153]:
df_top_n_read.set_index(['user_id'], inplace = True)

In [154]:
df_top_n_read.iloc[0]

r0    1152288
r1    1516481
r2      87632
r3     111408
r4     288617
r5      84117
r6    1510002
r7      98726
r8    2322597
r9      93614
Name: 1005ABAA83773778AB2A4BAAFE443901, dtype: int64

In [155]:
df_top_n_read.index

Index(['1005ABAA83773778AB2A4BAAFE443901', 'FD711E1ED5A2DA3F69425CDD77A80E5A',
       'A12D74E5CFC1564773D7C48B490DBE54', '3D693454C428B1D6E5458ACC210F9695',
       '771E3C17495EE9D080AF87EB2504161F', '7788F6DEA6D525F6A79EC6208AE8C793',
       '4F127FBECCC472FC8109EE9DA46FB8BE', '6B525E7772D5BCEE61CC6A282448B360',
       'A3BEC98880E7420DE9B7D59932C9B75D', '29491F39069C9B01A593F4C600F13F4E',
       ...
       '56CD40E3365FE8712D8220639896C203', 'B5B8B54A7E9C4ED3E652AFC8B2D94A2B',
       '5AB42DB530F1F7C414D239F39D92E1AB', '51C55763B7314F20B3C209B33F0B725D',
       'CBF58BB5CBCB3282CCE7FCE281ED5352', 'BEDA3909A6EE531E14353DF03C415AF3',
       'BF7773D1C56DF2F66F695DF660EDD70E', '86AA28E95FF56349EB9B4309E292C294',
       '9EC19CA14621FBFFF3A486BEDEF0876D', 'E810C7DE395924F731A5E445D4B52D5C'],
      dtype='object', name='user_id', length=7194)

In [163]:
list(df_top_n_read.loc['1005ABAA83773778AB2A4BAAFE443901'])

[1152288,
 1516481,
 87632,
 111408,
 288617,
 84117,
 1510002,
 98726,
 2322597,
 93614]

In [282]:
df_offering = pd.read_csv('/Users/albinasitdikova/Project_5/df_offering.csv')

In [176]:
type(df_offering.id[0])

numpy.int64

In [177]:
df_offering['id'] = df_offering['id'].astype(str)

In [179]:
df_offering[df_offering.id == '1152288']

,hotel_class,region_id,url,phone,details,type,id,name,region,street-address,postal-code,locality
3808,5.0,60878,http://www.tripadvisor.com/Hotel_Review-g60878...,NaN,NaN,hotel,1152288,Four Seasons Hotel Seattle,WA,99 Union Street,NaN,Seattle


In [181]:
df_offering.url[df_offering.id == '1152288']

3808    http://www.tripadvisor.com/Hotel_Review-g60878...
Name: url, dtype: object